<a href="https://colab.research.google.com/github/lorenafc/Capita_Selecta_RHI50403/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install googlemaps -q

  Preparing metadata (setup.py) ... done


In [2]:
#import libraries
import pandas as pd
import geopandas as gpd
import requests  # To make API requests
import html  # Import the html module for decoding HTML entities
from geopy.geocoders import Nominatim
from geopy.exc import GeopyError
import time
import logging
from tqdm import tqdm # to show the percentage of the geocoding process for each city
from geopandas.tools import geocode
import googlemaps
from googlemaps import Client as GoogleMaps

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# upload the file
from google.colab import files
uploaded=files.upload()

Saving additional_authors.xlsx to additional_authors.xlsx


In [6]:
file_name = 'additional_authors.xlsx'

# Read the Excel file
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')

authors_small_cities_copy = authors_small_cities.copy()

The geocoging takes too long to run, and it crashes if you run the whole file in Google Colab. So, in Colab I divided the code below it in subsets of 10.000 rows (and still, it took more than 3h to run each subset). After the geocoding, I merged all the files to use it for the maps vizualization

In [40]:
smalldataset_ten_thousand = authors_small_cities # I added, for instance, (....).iloc[10000:20000,:] In the small example displayed below there are only 7 rows: .iloc[:7,:]


In [41]:
smalldataset_ten_thousand

,starturl,originalbirth,originaldeath,cleanedbirth,cleaneddeath,finalname,georeferenceurl,death,born,active
0,http://viaf.org/viaf/81048146,1570.0,1634.0,1570,1634.0,"Carneiro, Tomé Tavares 1570-1634",viafprocessedfile,NaN,,Rio Tinto
1,http://viaf.org/viaf/180581759,0.0,1775.0,NaN,1775.0,"Kolíček, Caesarij, -1775",viafprocessedfile,Cieszin,,NaN
2,http://viaf.org/viaf/100161519,0.0,1570.0,NaN,1570.0,"Hertel, Jakob 15..?-1570",http://catalogue.bnf.fr/ark:/12148/cb15011774s,NaN,(Chur,NaN
3,http://viaf.org/viaf/169350346,1699.0,1770.0,1699,1770.0,"Ferro Caaveyro, Lucas Antonio ca. 1699-1770",www.wikidata.org/wiki/Q3393623#sitelinks-wikip...,NaN,A Capela,NaN
4,http://viaf.org/viaf/67541728,1502.0,1600.0,1502,1600.0,"Aparicio, Sebastián de 1502-1600 Santo",www.wikidata.org/wiki/Q2486114#sitelinks-wikip...,Puebla City,A GudiÃ±a,NaN
5,http://viaf.org/viaf/121452165,1320.0,1394.0,1320,1394.0,"Kukullei, Janos",www.wikidata.org/wiki/Q1126408#sitelinks-wikip...,NaN,Å ariÅ¡skÃ© Sokolovce,NaN
6,http://viaf.org/viaf/62383957,1538.0,1592.0,1538,1592.0,"Bonifacio, Natale, 1538-1592",www.wikidata.org/wiki/Q3513974#sitelinks-wikip...,Šibenik,Å ibenik,NaN


#ENCODING FIX

(not all of the names were fixed)

In [42]:
#source: chatGPT and Lucas Koren

# keeping the libraries here too in comments to make it easier to verify the ones used for the encoding

# import pandas as pd
# import html

def setup_and_fix_encoding(df):
    """
    Creates necessary city columns if they don't exist and applies encoding fixes to city name columns.

    :param df: pd.DataFrame, the DataFrame containing the original city data
    :return: pd.DataFrame, the DataFrame with fixed encodings for city names
    """
    # Define the mappings from original columns to new city name columns if they don't exist
    column_mappings = {
        'born': 'borncity',
        'death': 'deathcity',
        'active': 'activecity'
    }

    # Create new city columns from existing data if they don't exist
    for original_col, new_col in column_mappings.items():
        if new_col not in df.columns:
            df[new_col] = df[original_col]

    # Define possible encodings
    encodings = ['utf-8', 'latin1', 'windows-1252', 'windows-1250', "latin2", "iso-8859-1", "iso-8859-2", 'iso-8859-15',
                 'macroman', 'ascii', 'cp1250', 'cp1251', 'cp1253', 'cp1254', 'cp1257', 'iso-8859-3', 'iso-8859-4',
                 'iso-8859-5', 'iso-8859-6', 'iso-8859-7', 'iso-8859-8', 'iso-8859-9', 'koi8-r', 'koi8-u']

    def fix_encoding(text):
        if not isinstance(text, str):
            return text
        # Decode HTML entities first
        text = html.unescape(text)
        # Try decoding and encoding combinations
        for enc1 in encodings:
            for enc2 in encodings:
                if enc1 != enc2:
                    try:
                        decoded_text = text.encode(enc2, errors='replace').decode(enc1, errors='replace')
                        if decoded_text.encode(enc1, errors='replace').decode(enc2, errors='replace') == text:
                            return decoded_text
                    except (UnicodeEncodeError, UnicodeDecodeError):
                        continue

        return text

    # Apply the encoding fix function to the newly created city name columns
    for _, new_col in column_mappings.items():
        df[new_col] = df[new_col].apply(fix_encoding)

    return df

# Example:

# smalldataset_ten_thousand_cleaned = setup_and_fix_encoding(smalldataset_ten_thousand)
# print(smalldataset_ten_thousand_cleaned)

                         starturl  originalbirth  originaldeath cleanedbirth  \
0   http://viaf.org/viaf/81048146         1570.0         1634.0         1570   
1  http://viaf.org/viaf/180581759            0.0         1775.0          NaN   
2  http://viaf.org/viaf/100161519            0.0         1570.0          NaN   
3  http://viaf.org/viaf/169350346         1699.0         1770.0         1699   
4   http://viaf.org/viaf/67541728         1502.0         1600.0         1502   
5  http://viaf.org/viaf/121452165         1320.0         1394.0         1320   
6   http://viaf.org/viaf/62383957         1538.0         1592.0         1538   

   cleaneddeath                                    finalname  \
0        1634.0             Carneiro, Tomé Tavares 1570-1634   
1        1775.0                     Kolíček, Caesarij, -1775   
2        1570.0                     Hertel, Jakob 15..?-1570   
3        1770.0  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4        1600.0       Aparicio, Sebasti

<ipython-input-42-cc7cb18baf22>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col] = df[original_col]
<ipython-input-42-cc7cb18baf22>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col] = df[original_col]
<ipython-input-42-cc7cb18baf22>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [43]:
def clean_and_rename_dataset(df):
    """
    Renames columns in the DataFrame, drops unnecessary columns, and reorders the remaining columns.

    :param df: pd.DataFrame, the original DataFrame to be cleaned and renamed
    :return: pd.DataFrame, the cleaned and reordered DataFrame
    """
    # Rename the columns
    df.rename(columns={'cleanedbirth': 'birthyear',
                       'cleaneddeath': 'deathyear',
                       'finalname': 'nameandbirthdeathyear'},
              inplace=True)

    # Drop unnecessary columns
    df_cleaned = df.drop(columns=["originalbirth", "originaldeath"])

    # Reorder the columns
    df_cleaned = df_cleaned[['starturl', 'birthyear', 'deathyear', 'nameandbirthdeathyear',
                             'georeferenceurl', 'born', 'borncity', 'death',
                             'deathcity', 'active', 'activecity']]

    return df_cleaned

# Example:

# smalldataset_ten_thousand_cleaned = clean_and_rename_dataset(smalldataset_ten_thousand)
# print(smalldataset_ten_thousand_cleaned)

# export the result as excel file
# file_name = 'additional_authors_encoded.xlsx'
# smalldataset_ten_thousand_cleaned.to_excel(file_name)


                         starturl birthyear  deathyear  \
0   http://viaf.org/viaf/81048146      1570     1634.0   
1  http://viaf.org/viaf/180581759       NaN     1775.0   
2  http://viaf.org/viaf/100161519       NaN     1570.0   
3  http://viaf.org/viaf/169350346      1699     1770.0   
4   http://viaf.org/viaf/67541728      1502     1600.0   
5  http://viaf.org/viaf/121452165      1320     1394.0   
6   http://viaf.org/viaf/62383957      1538     1592.0   

                         nameandbirthdeathyear  \
0             Carneiro, Tomé Tavares 1570-1634   
1                     Kolíček, Caesarij, -1775   
2                     Hertel, Jakob 15..?-1570   
3  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4       Aparicio, Sebastián de 1502-1600 Santo   
5                              Kukullei, Janos   
6                 Bonifacio, Natale, 1538-1592   

                                     georeferenceurl                   born  \
0                                  viafprocessedfile   

<ipython-input-43-c22e2c52d447>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'cleanedbirth': 'birthyear',


#GEOCODING

There are two possibilities of geocoding displayed below: using Nominatim (free) and google API (free of charge, but it is paid after reach a certain usage limit)

## Nominatim

In [13]:
#source: chatGPT and Lucas Koren

# keeping the libraries here too in comments to make it easier to verify the ones used for the geocoding with Nominatim

# import logging
# import time
# import pandas as pd
# from geopy.geocoders import Nominatim
# from tqdm import tqdm

def full_geocoding_process(df, output_file_name):
    """
    Geocodes cities listed in the DataFrame and maps the coordinates back to the DataFrame. Saves the DataFrame as an Excel file.

    :param df: pd.DataFrame, the DataFrame containing city names to geocode
    :param output_file_name: str, the filename for the output Excel file
    :return: pd.DataFrame, the DataFrame with new columns for geocoded coordinates
    """
    # Initialize logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Initialize Nominatim geocoder with a user agent
    geolocator = Nominatim(user_agent="my_Geocoder_application")

    # Function to geocode city names with error handling and rate limiting
    def geocode_city(city_name):
        try:
            # Rate limiting
            time.sleep(1)
            # Attempt to geocode the city name with a timeout
            location = geolocator.geocode(city_name)
            if location:
                return (city_name, location.latitude, location.longitude)
            else:
                logging.warning(f"Geocoding failed for: {city_name}")
                return (city_name, None, None)
        except Exception as e:
            logging.error(f"Geocoding error for {city_name}: {e}")
            return (city_name, None, None)

    # Extract unique city names from all columns
    unique_cities = list(set(df['borncity'].dropna().tolist() +
                             df['deathcity'].dropna().tolist() +
                             df['activecity'].dropna().tolist()))

    # Geocode unique city names with progress tracking
    geocoded_results = [geocode_city(city) for city in tqdm(unique_cities, desc='Geocoding cities')]

    # Create a dictionary to map city names to their geocoded coordinates
    geocode_dict = {result[0]: (result[1], result[2]) for result in geocoded_results}

    # Function to map geocoded coordinates to a column in the DataFrame
    def map_coordinates(column_name, lat_col_name, lon_col_name):
        # Map geocoded coordinates to the original DataFrame
        df.loc[:, f'{column_name}_coordinates'] = df[column_name].map(geocode_dict)
        # Ensure coordinates are always tuples with two elements
        df.loc[:, f'{column_name}_coordinates'] = df[f'{column_name}_coordinates'].apply(lambda x: (x if isinstance(x, tuple) else (None, None)))

        # Extract latitude and longitude from coordinates, filling NaN where necessary
        df[[lat_col_name, lon_col_name]] = df[f'{column_name}_coordinates'].apply(pd.Series)

        # Drop the coordinates column
        df.drop(columns=[f'{column_name}_coordinates'], inplace=True)

    # Map geocoded coordinates to activecity, borncity, and deathcity columns with distinct names for latitude and longitude
    columns_to_map = [
        ('borncity', 'latitude_born', 'longitude_born'),
        ('deathcity', 'latitude_death', 'longitude_death'),
        ('activecity', 'latitude_active', 'longitude_active')
    ]

    for column, lat_col, lon_col in columns_to_map:
        map_coordinates(column, lat_col, lon_col)

    # drop columns that are no longer needed
    df.drop(['born', 'death', "active"], axis=1, inplace=True)

    # Save the DataFrame as an Excel file
    df.to_excel(output_file_name, index=False)

    return df

# Example usage
# smalldataset_ten_thousand_cleaned = full_geocoding_process(smalldataset_ten_thousand_cleaned, 'smalldataset_nominatim.xlsx')
# print(smalldataset_ten_thousand_cleaned.head())

Geocoding cities: 100%|██████████| 22/22 [00:27<00:00,  1.27s/it]

                         starturl birthyear  deathyear  \
0   http://viaf.org/viaf/81048146      1570     1634.0   
1  http://viaf.org/viaf/180581759       NaN     1775.0   
2  http://viaf.org/viaf/100161519       NaN     1570.0   
3  http://viaf.org/viaf/169350346      1699     1770.0   
4   http://viaf.org/viaf/67541728      1502     1600.0   

                         nameandbirthdeathyear  \
0             Carneiro, Tomé Tavares 1570-1634   
1                     Kolíček, Caesarij, -1775   
2                     Hertel, Jakob 15..?-1570   
3  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4       Aparicio, Sebastián de 1502-1600 Santo   

                                     georeferenceurl  borncity    deathcity  \
0                                  viafprocessedfile                    NaN   
1                                  viafprocessedfile                Cieszin   
2     http://catalogue.bnf.fr/ark:/12148/cb15011774s     (Chur          NaN   
3  www.wikidata.org/wiki/Q3393623#

## Google API

In [ ]:
# pip install googlemaps -q

In [ ]:
# keeping the libraries here too in comments to make it easier to verify the ones used for the geocoding with Google API

# import time
# import logging
# import pandas as pd
# from tqdm import tqdm
# from googlemaps import Client as GoogleMaps


In [48]:
#source: ChatGPT and Lucas Koren

def geocode_and_update_df(df, gmaps_key, file_name):
    """
    Geocodes city names in specified DataFrame columns using Google Maps API, updates DataFrame with latitude and longitude,
    and saves the updated DataFrame to an Excel file.

    :param df: pd.DataFrame, the DataFrame to process.
    :param gmaps_key: str, the API key for Google Maps.
    :param file_name: str, the filename for the output Excel file.
    :return: pd.DataFrame, the updated DataFrame with new latitude and longitude columns.
    """
    # Check if the DataFrame is None and log an error if true, then exit the function
    if df is None:
        logging.error("Input DataFrame is None.")
        return None

    # Setup logging configuration to display timestamp, severity, and message of log entries
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Initialize Google Maps API client with the provided API key
    gmaps = GoogleMaps(gmaps_key)

    def geocode_city(city_name):
        # Rate limit the geocoding requests to avoid hitting API limits
        time.sleep(1)
        try:
            # Perform the geocoding request for a city name
            geocode_result = gmaps.geocode(city_name)
            # If geocoding is successful, extract and return the latitude and longitude
            if geocode_result:
                location = geocode_result[0]['geometry']['location']
                return (city_name, location['lat'], location['lng'])
            else:
                # Log a warning if geocoding fails but no exception was thrown
                logging.warning(f"Geocoding failed for: {city_name}")
                return (city_name, None, None)
        except Exception as e:
            # Log any exceptions encountered during geocoding
            logging.error(f"Geocoding error for {city_name}: {e}")
            return (city_name, None, None)

    def geocode_column(column_name, lat_col_name, lon_col_name):
        # Extract unique non-null city names from the specified column to avoid redundant API calls
        unique_cities = list(set(df.dropna(subset=[column_name])[column_name]))
        # Geocode each city and collect results
        geocoded_results = [geocode_city(city) for city in tqdm(unique_cities, desc=f'Geocoding {column_name}')]
        # Create a dictionary to map city names to their coordinates
        geocode_dict = {result[0]: (result[1], result[2]) for result in geocoded_results}
        # Map the geocoded coordinates back to the original DataFrame
        df[f'{column_name}_coordinates'] = df[column_name].map(geocode_dict)
        # Ensure all entries are tuples (handles missing or failed geocodings)
        df[f'{column_name}_coordinates'] = df[f'{column_name}_coordinates'].apply(lambda x: (x if isinstance(x, tuple) else (None, None)))
        # Split the coordinates into separate latitude and longitude columns
        df[[lat_col_name, lon_col_name]] = df[f'{column_name}_coordinates'].apply(pd.Series)
        # Remove the temporary coordinates column
        df.drop(columns=[f'{column_name}_coordinates'], inplace=True)

    # List of columns to geocode along with their corresponding names for latitude and longitude columns
    columns_to_geocode = [
        ('borncity', 'latitude_born', 'longitude_born'),
        ('deathcity', 'latitude_death', 'longitude_death'),
        ('activecity', 'latitude_active', 'longitude_active')
    ]

    # Process each column to geocode as specified in the list
    for column, lat_col, lon_col in columns_to_geocode:
        geocode_column(column, lat_col, lon_col)

    # Save the updated DataFrame to an Excel file
    df.to_excel(file_name, index=False)
    # Return the modified DataFrame
    return df

# Example:

# gmaps_key = 'YOUR_GOOGLE_MAPS_API_KEY'  # Replace with your actual API key
# file_name = 'smalldataset_GoogleAPI.xlsx'
# smalldataset_ten_thousand_cleaned = geocode_and_update_df(smalldataset_ten_thousand_cleaned, gmaps_key, file_name)


Geocoding activecity: 100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


In [49]:
# print(smalldataset_ten_thousand_cleaned)

                         starturl birthyear  deathyear  \
0   http://viaf.org/viaf/81048146      1570     1634.0   
1  http://viaf.org/viaf/180581759       NaN     1775.0   
2  http://viaf.org/viaf/100161519       NaN     1570.0   
3  http://viaf.org/viaf/169350346      1699     1770.0   
4   http://viaf.org/viaf/67541728      1502     1600.0   
5  http://viaf.org/viaf/121452165      1320     1394.0   
6   http://viaf.org/viaf/62383957      1538     1592.0   

                         nameandbirthdeathyear  \
0             Carneiro, Tomé Tavares 1570-1634   
1                     Kolíček, Caesarij, -1775   
2                     Hertel, Jakob 15..?-1570   
3  Ferro Caaveyro, Lucas Antonio ca. 1699-1770   
4       Aparicio, Sebastián de 1502-1600 Santo   
5                              Kukullei, Janos   
6                 Bonifacio, Natale, 1538-1592   

                                     georeferenceurl            borncity  \
0                                  viafprocessedfile      

# Merging the excel files

If you use Google Colab, you needed to make subsets to geocode the whole dataset (Colab will crash if you try to geocode all at once). To merge all the geocoded files, use the function below

In [ ]:
def merge_excel_files(base_path, file_names, output_file_name):
    """
    Merges multiple Excel files into a single DataFrame and saves it as a new Excel file.

    param base_path: str, the base path where the Excel files are located
    param file_names: list, the list of file names to be merged
    param output_file_name: str, the name of the output Excel file to be saved

    return: None
    """
    # Read each file into a DataFrame and store them in a list
    dataframes = [pd.read_excel(base_path + file_name) for file_name in file_names]

    # Concatenate all DataFrames into one, ignoring the original index
    merged_dataframe = pd.concat(dataframes, ignore_index=True)

    # Save the merged DataFrame to a new Excel file in your Google Drive
    merged_dataframe.to_excel(base_path + output_file_name, index=False)

# Example

# base_path = '/content/drive/My Drive/'  # location of the geocoded excel files
# file_names = [
#     '1_smalldataset.xlsx',
#     '2_smalldataset.xlsx',
#     '3_smalldataset.xlsx',
#     '4_smalldataset.xlsx',
#     '5_smalldataset.xlsx',
#     '6_smalldataset.xlsx',
#     '7_smalldataset.xlsx',
#     '8_smalldataset.xlsx'
# ]

# output_file_name = 'merged_additional_authors_encoded.xlsx'

# merge_excel_files(base_path, file_names, output_file_name)
